In [1]:
import ee
import geemap
# 不再需要 import sys, import os (除非其他地方用到)
# 也不再需要任何 sys.path.append(...)

# 初始化GEE
ee.Initialize(project='geemap-441216') # 替换为您的项目ID

# Python会自动在当前文件夹下寻找模块，所以可以直接导入！
try:
    import z_flood_robust
    from z_flood_robust import zscore # 甚至可以这样导入
    zscore.calc_basemad # 尝试访问
    print("本地自定义模块 z_flood_robust 已成功导入！")
except ImportError as e:
    print(f"导入模块失败: {e}")
    print("请确保'z_flood_robust'文件夹与您的Jupyter笔记本在同一个目录下。")

本地自定义模块 z_flood_robust 已成功导入！


# 1.导入依赖-数据管理模块

In [2]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle #用于在图表中添加形状(例如矩形)
import re #正则表达式模块
from z_flood_robust import calc_basemedian, calc_basemad, calc_median_anomaly, calc_robust_zscore, calc_basemean,calc_basestd,calc_zscore
from z_flood_robust import mapFloods,floodPalette

from ipywidgets import Label #用于创建交互式控件



# 2.定义交互式地图以选择感兴趣区域-界面与交互模块

In [12]:
#解析用户点击地图时生成的坐标字符串，提取经纬度并返回
def parseClickedCoordinates(label):
  #利用正则表达式，从label.value提取出坐标：一个可能带有负号的浮点数 这里为【经度，纬度】
  #正则表达式： r'(?:-)?[0-9]+.[0-9]+'
  #r'表示原始字符，(?:-)是非捕获组，?:表示负号是可选的，[0-9]+表示匹配一个数字，+表示可以出现一次或多次，.匹配小数点，最后再次匹配一个或多个数字
  coords = [float(c) for c in re.findall(r'(?:-)?[0-9]+.[0-9]+', label.value)]
  coords.reverse() #反转为【纬度，经度】，符合GEE坐标格式
  return coords

#创建一个Lable，用于显示用户点击的目标
l = Label()
display(1)
#处理用户与地图的交互事件，当用户点击地图时，将点击的坐标存储到Label控件中
def handle_interaction(**kwargs):
  #kwargs包含交互事件的参数 kwargs.get('type'):获取事件类型为鼠标点击 kwargs.get('coordinates')：获取点击的坐标，转换为字符串并存储到Label控件中
  if kwargs.get('type') == 'click':
    l.value = str(kwargs.get('coordinates'))

print('请点击地图以选择你要监测的区域')
#创建交互式地图
Map = geemap.Map()
Map.on_interaction(handle_interaction)
Map

1

请点击地图以选择你要监测的区域


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

# 3.定义几何范围并展示-界面与交互模块

## 3.1.手动划定研究区域

In [13]:
lon,lat = parseClickedCoordinates(l)
w,h = 2,2 #矩形宽度与高度（单位：度）

geometry = ee.Geometry.Polygon(
    [[[lon-w,lat-h],
     [lon-w,lat+h],
     [lon+w,lat+h],
     [lon+w,lat-h]]]
)

#将几何范围添加到地图
Map.addLayer(
    geometry,
    {'color':'red','fillColor':'00000000'},
    'AOI'
)
Map

Map(bottom=399611.0, center=[39.38139089214383, -0.3865922574188186], controls=(WidgetControl(options=['positi…

In [14]:
#USE MAP RECTANGLE 选择区域
#获取感兴趣区域（用于论文绘图)
roi_choose1 = Map.user_roi

if roi_choose1 is not None:
  #获取ROI类型
  roi_type = roi_choose1.type().getInfo()
  print(f"ROI 类型：{roi_type}")

  #如果是Polygon，获取坐标
  if roi_type == 'Polygon':
    coords = roi_choose1.coordinates().getInfo()
    print(f"ROI 坐标：{coords}")

ROI 类型：Polygon
ROI 坐标：[[[-0.424067, 39.459907], [-0.424067, 39.468588], [-0.414195, 39.468588], [-0.414195, 39.459907], [-0.424067, 39.459907]]]


## 3.2.从GEE资产里加载研究区

In [11]:
asset_id = 'projects/geemap-441216/assets/valencia_final_aoi'

try:
    # 从GEE Assets加载矢量文件作为FeatureCollection
    aoi_fc = ee.FeatureCollection(asset_id)
    
    
    # 对于洪水分析，我们通常需要的是所有要素合并后的几何边界
    # .geometry() 会将FeatureCollection中所有的要素融合成一个单一的几何对象
    roi_choose1 = aoi_fc.geometry()
    
    print("成功从GEE Assets加载研究区！")
    
    # 创建一个地图来可视化你的AOI，确保加载正确
    Map = geemap.Map()
    Map.addLayer(roi_choose1, {'color': 'red', 'fillColor': '00000000'}, 'Study Area (from Asset)')
    Map.centerObject(roi_choose1, 10) # 自动缩放到研究区
    display(Map)
    
except ee.EEException as e:
    print(f"加载AOI失败！请检查你的Asset ID是否正确: {asset_id}")
    print(f"错误信息: {e}")
    # 如果加载失败，停止执行后续代码
    raise SystemExit("无法加载AOI，程序终止。")
    

成功从GEE Assets加载研究区！


Map(center=[39.41956382721997, -0.431725495619315], controls=(WidgetControl(options=['position', 'transparent_…

# 4 设置日期，过滤影像计算数量（以下是西班牙巴伦西亚区域）

In [15]:
targdate = '2024-10-31'   # 目标日期 (需要根据实际数据可用性验证或微调)
basestart = '2023-11-20'  # 基线开始日期
baseend = '2024-09-10'   # 基线结束日期

# !!! (可选) 替换硬编码的 aoi2 !!!
# 如果你决定使用固定的 valencia_aoi，在这里替换：
#aoi3 = ee.Geometry.Rectangle([-0.55, 39.35, -0.30, 39.50]) # 新的巴伦西亚AOI
# !!! 更好的做法是完全不用 aoi2，下面 filterBounds 直接用 analysis_aoi !!!

filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation", "VV"),
    ee.Filter.listContains("transmitterReceiverPolarisation", "VH"), # 如果需要VH也取消注释
    ee.Filter.equals("instrumentMode", "IW"),
    # !!! 重要：检查轨道方向 !!!
    # 需要确认 targdate 前后过境巴伦西亚的 S1 影像是升轨还是降轨
    # 可能需要改为 'DESCENDING'，或不加这个过滤器以获取所有轨道
    ee.Filter.equals("orbitProperties_pass", "ASCENDING"), # <--- 可能需要修改或移除!
    # 时间过滤器现在是动态的，覆盖基线期和目标期，无需修改
    ee.Filter.date('2022-08-21', ee.Date(targdate).advance(1, 'day'))
]

# 加载S1数据并计算Z分数
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filter(filters) \
    .filterBounds(roi_choose1) # !!! 使用统一的 analysis_aoi !!!



# 检查是否有影像满足条件
s1_size = s1.size().getInfo()
print(f"找到满足条件的 Sentinel-1 影像数量: {s1_size}")
if s1_size == 0:
    print("警告：未找到满足条件的影像，请检查时间范围、AOI 或轨道方向过滤器！")
print(s1.aggregate_array('system:time_start').map(lambda t: ee.Date(t).format()).getInfo())

# Z 分数计算 (轨道方向参数需要与上面 filter 保持一致)
# orbit_direction = "ASCENDING" # <--- 如果上面修改了这里也要改
orbit_direction = "ASCENDING" if ee.Filter.equals("orbitProperties_pass", "ASCENDING") in filters else "DESCENDING" #或者更灵活的方式
z = calc_zscore(s1, basestart, baseend, 'IW', orbit_direction)
rz = calc_robust_zscore(s1, basestart, baseend, 'IW', orbit_direction)


MAD = calc_basemad(s1, basestart, baseend, 'IW', orbit_direction)

找到满足条件的 Sentinel-1 影像数量: 133
['2022-08-26T17:54:58', '2022-08-31T18:02:53', '2022-09-07T17:54:59', '2022-09-12T18:02:54', '2022-09-19T17:54:59', '2022-09-24T18:02:54', '2022-10-01T17:54:59', '2022-10-06T18:02:54', '2022-10-13T17:54:59', '2022-10-25T17:55:00', '2022-10-30T18:02:54', '2022-11-06T17:54:59', '2022-11-11T18:02:54', '2022-11-18T17:54:59', '2022-11-23T18:02:54', '2022-11-30T17:54:58', '2022-12-05T18:02:53', '2022-12-12T17:54:58', '2022-12-17T18:02:52', '2022-12-24T17:54:57', '2022-12-29T18:02:52', '2023-01-05T17:54:57', '2023-01-10T18:02:51', '2023-01-17T17:54:56', '2023-01-22T18:02:51', '2023-01-29T17:54:56', '2023-02-03T18:02:51', '2023-02-10T17:54:55', '2023-02-15T18:02:50', '2023-02-22T17:54:55', '2023-02-27T18:02:50', '2023-03-06T17:54:55', '2023-03-11T18:02:50', '2023-03-18T17:54:55', '2023-03-23T18:02:50', '2023-03-30T17:54:55', '2023-04-04T18:02:50', '2023-04-11T17:54:56', '2023-04-16T18:02:51', '2023-04-23T17:54:56', '2023-04-28T18:02:51', '2023-05-05T17:54:57', '202

# 5.对照方法

## 5.1 全局OTSU阈值结合JRC永久水体掩膜修正

In [16]:
# --- 1. 定义您提供的Otsu函数 ---
# (这是您在原始代码中编写的、在GEE服务器端运行的Otsu算法)
def otsu(histogram):
  histogram = ee.Dictionary(histogram);
  counts = ee.Array(histogram.get('histogram'))
  means = ee.Array(histogram.get('bucketMeans'))
  size = means.length().get([0])
  total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
  sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
  mean = sum.divide(total)
  
  indices = ee.List.sequence(1, size)
  
  def compute_bss(i):
    aCounts = counts.slice(0, 0, i)
    aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
    aMeans = means.slice(0, 0, i)
    aMean = aMeans.multiply(aCounts).reduce(ee.Reducer.sum(), [0]).get([0]).divide(aCount)
    bCount = total.subtract(aCount)
    bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
    return aCount.multiply(aMean.subtract(mean).pow(2)).add(bCount.multiply(bMean.subtract(mean).pow(2)))

  bss = indices.map(compute_bss)
  
  return means.sort(bss).get([-1])

In [17]:

# --- 2. 获取洪水目标日的SAR影像 ---
target_date_ee = ee.Date(targdate)
s1_target_collection = s1.filterDate(target_date_ee, target_date_ee.advance(1, 'day'))

target_image_size = s1_target_collection.size().getInfo()
if target_image_size == 0:
    print(f"错误：在目标日期 {targdate} 未找到任何Sentinel-1影像，无法执行Otsu法。")
else:
    print(f"在 {targdate} 找到 {target_image_size} 景影像，将合并后进行处理。")
    kernel_radius = 2
    sar_image_for_otsu = s1_target_collection.select('VV').mosaic().clip(roi_choose1)\
                                             .focal_mean(radius = kernel_radius, units='pixels')

    # --- 3. 加载JRC永久水体数据并创建掩膜 ---
    print("加载JRC全球地表水数据...")
    jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
    
    # 核心修正
    # 使用.unmask(0)将所有无数据的像元(如陆地)赋值为0
    occurrence_unmasked = jrc_water.select('occurrence').unmask(0)
    
    permanent_water_mask = occurrence_unmasked.gt(90)
    print("永久性水体掩膜创建完成 (occurrence > 90%)。")

    # --- 4. 计算Otsu阈值并提取初始水体 ---
    print("正在计算影像直方图...")
    histogram_dict = sar_image_for_otsu.reduceRegion(
        reducer=ee.Reducer.histogram(255, 0.1), # buckets, maxBuckets
        geometry=roi_choose1,
        scale=30,
        maxPixels=1e10
    )
    
    # !!! 核心修正：调用您自己定义的 otsu 函数 !!!
    otsu_threshold = otsu(histogram_dict.get('VV'))
    
    otsu_threshold_value = otsu_threshold.getInfo()
    print(f"计算出的全局Otsu阈值为: {otsu_threshold_value:.4f} dB")
    
    initial_water_otsu = sar_image_for_otsu.lt(otsu_threshold_value)

    # --- 5. 移除永久性水体，得到最终洪水范围 ---
    flood_map_otsu = initial_water_otsu.And(permanent_water_mask.Not())
    print("已从Otsu结果中移除永久性水体，生成最终洪水图。")

    # --- 6. 可视化结果 ---
    print("正在准备可视化地图...")
    Map_Otsu = geemap.Map()
    Map_Otsu.centerObject(roi_choose1, 11)

    sar_vis_params = {'min': -25, 'max': 0}
    water_vis_params = {'palette': ['#0000FF']}

    Map_Otsu.addLayer(sar_image_for_otsu, sar_vis_params, f'SAR Image ({targdate})')
    Map_Otsu.addLayer(permanent_water_mask.selfMask(), {'palette': ['#CCCCCC']}, 'JRC Permanent Water (>90%)')
    Map_Otsu.addLayer(initial_water_otsu.selfMask(), {'palette': ['#FFFF00']}, 'Initial Water (Otsu)')
    Map_Otsu.addLayer(flood_map_otsu.selfMask(), water_vis_params, 'Final Flood (Otsu - JRC)')
    
    display(Map_Otsu)

在 2024-10-31 找到 1 景影像，将合并后进行处理。
加载JRC全球地表水数据...
永久性水体掩膜创建完成 (occurrence > 90%)。
正在计算影像直方图...
计算出的全局Otsu阈值为: -10.4616 dB
已从Otsu结果中移除永久性水体，生成最终洪水图。
正在准备可视化地图...


Map(center=[39.464247445638335, -0.4191310000180092], controls=(WidgetControl(options=['position', 'transparen…

## 5.2 采用K-Means方法监督

In [21]:
sar_image_dual = s1_target_collection.select(['VV','VH']).mosaic().clip(roi_choose1)

# --- 1. 特征工程：计算比值和和值并合并波段 ---
print("正在计算 VV/VH 比值和 VV+VH 和值特征...")
    
# 提取VV和VH波段，方便进行计算
vv_band = sar_image_dual.select('VV')
vh_band = sar_image_dual.select('VH')
    
# 计算比值 (Ratio): VV / VH
# 比值可以放大水体和部分植被的差异
ratio_band = vv_band.divide(vh_band).rename('ratio')
    
# 计算和值 (Sum): VV + VH
# 和值可以综合反映总体的后向散射能量
sum_band = vv_band.add(vh_band).rename('sum')
    
# 将四个特征波段合并成一个多维特征影像
feature_image = ee.Image.cat([vv_band, vh_band, ratio_band, sum_band])
    
print("4维特征影像创建完成: [VV, VH, ratio, sum]")

# --- 2. K-Means聚类 ---
num_clusters = 5 # 对于4维特征，聚类数为5
num_samples = 5000
print(f"正在从特征影像中随机采样 {num_samples} 个点用于训练...")
training_samples = feature_image.sample(
        region=roi_choose1,
        scale=10,
        numPixels=num_samples,
        seed=0 # 添加一个随机种子确保每次运行结果一致
    )

# 初始化并训练K-Means聚类器
clusterer = ee.Clusterer.wekaKMeans(num_clusters).train(training_samples)

# 使用训练好的聚类器对整个特征影像分类
print("正在对整个研究区进行分类...")
clustered_image = feature_image.cluster(clusterer)

# --- 3.自动识别水体类别 ---
# 基于水体聚类VV平均值最低的原则，识别水体类
print("正在识别水体类别")

# 创建一个图像，波段1是每个像元VV值，波段2是聚类ID，这样可以统计哪个id对应的聚类平均VV最低
sort_image = feature_image.select('VV').addBands(clustered_image)

# 根据聚类id统计各类别VV均值
# 得到的是一个列表:{cluster:1,mean:-19.7}这样的
means_dict = sort_image.reduceRegion(
    reducer = ee.Reducer.mean().group(groupField=1 , groupName='cluster'),
    geometry = roi_choose1,
    scale = 30,
    maxPixels = 1e9
)

# 在客户端寻找mean值最小的类别
groups = ee.List(means_dict.get('groups')).getInfo()
min_mean = float('inf') #初始值设置为无穷大
water_cluster_id = -1
# 冒泡排序
for group in groups:
    if group['mean'] < min_mean:
        min_mean = group['mean']
        water_cluster_id = group['cluster']

if water_cluster_id != -1:
    print(f"识别完成！水体类别ID为: {water_cluster_id} (平均VV值为: {min_mean:.4f})")
    initial_water_kmeans = clustered_image.eq(water_cluster_id)
else:
    print("错误：未能成功识别水体类别。")
    # 创建一个空影像以避免后续代码出错
    initial_water_kmeans = ee.Image(0).selfMask()

 # --- 5. 移除永久性水体 ---
jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
occurrence_unmasked = jrc_water.select('occurrence').unmask(0)
permanent_water_mask = occurrence_unmasked.gt(90)
    
flood_map_kmeans = initial_water_kmeans.And(permanent_water_mask.Not())
print("已从K-Means结果中移除永久性水体，生成最终洪水图。")

# --- 6. 可视化结果 ---
print("正在准备可视化地图...")
Map_KMeans_New = geemap.Map()
Map_KMeans_New.centerObject(roi_choose1, 11)

sar_vis_params = {'min': -25, 'max': 0}
water_vis_params = {'palette': ['#0000FF']}
cluster_palette = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF']
cluster_vis_params = {'min': 0, 'max': num_clusters - 1, 'palette': cluster_palette[:num_clusters]}

Map_KMeans_New.addLayer(sar_image_dual.select('VV'), sar_vis_params, f'SAR Image VV ({targdate})')
Map_KMeans_New.addLayer(clustered_image, cluster_vis_params, 'K-Means Clusters (Ratio/Sum)')
Map_KMeans_New.addLayer(initial_water_kmeans.selfMask(), {'palette': ['#FFFF00']}, 'Initial Water (K-Means)')
Map_KMeans_New.addLayer(flood_map_kmeans.selfMask(), water_vis_params, 'Final Flood (K-Means Ratio/Sum - JRC)')

Map_KMeans_New
    





正在计算 VV/VH 比值和 VV+VH 和值特征...
4维特征影像创建完成: [VV, VH, ratio, sum]
正在从特征影像中随机采样 5000 个点用于训练...
正在对整个研究区进行分类...
正在识别水体类别
识别完成！水体类别ID为: 2 (平均VV值为: -20.1734)
已从K-Means结果中移除永久性水体，生成最终洪水图。
正在准备可视化地图...


Map(center=[39.41956382721997, -0.431725495619315], controls=(WidgetControl(options=['position', 'transparent_…

## 5.3 Z分数固定阈值分割洪水



In [17]:
z_score_image = z.map(lambda image: image.clip(roi_choose1)).mosaic().select('VV')

Z_THRESHOLD = -2.5

print(f"正在应用固定的Z-Score阈值: Z < {Z_THRESHOLD}")
    
# 应用阈值：Z-Score值低于阈值的像元被分类为水体 (值为1)
initial_water_zscore = z_score_image.lt(Z_THRESHOLD)

# --- 3. 移除永久性水体 ---
# (此逻辑与其他对照方法完全相同)
jrc_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')
occurrence_unmasked = jrc_water.select('occurrence').unmask(0)
permanent_water_mask = occurrence_unmasked.gt(90)
    
flood_map_zscore = initial_water_zscore.And(permanent_water_mask.Not())
print("已从Z-Score阈值结果中移除永久性水体，生成最终洪水图。")

# --- 4. 可视化结果 ---
print("正在准备可视化地图...")
Map_ZScore_Threshold = geemap.Map()
Map_ZScore_Threshold.centerObject(roi_choose1, 11)

# Z-Score影像的可视化参数 (使用你论文中统一的配色)
zpalette = ['#b2182b','#ef8a62','#fddbc7','#f7f7f7','#d1e5f0','#67a9cf','#2166ac']
z_score_vis_params = {'min': -5, 'max': 5, 'palette': zpalette}
    
# 水体的可视化参数
water_vis_params = {'palette': ['#0000FF']}

# 添加图层以便对比
Map_ZScore_Threshold.addLayer(z_score_image, z_score_vis_params, f'Z-Score Image ({targdate})')
Map_ZScore_Threshold.addLayer(initial_water_zscore.selfMask(), {'palette': ['#FFFF00']}, f'Initial Water (Z < {Z_THRESHOLD})')
Map_ZScore_Threshold.addLayer(flood_map_zscore.selfMask(), water_vis_params, 'Final Flood (Z-Score Threshold - JRC)')
    
Map_ZScore_Threshold

正在应用固定的Z-Score阈值: Z < -2.5
已从Z-Score阈值结果中移除永久性水体，生成最终洪水图。
正在准备可视化地图...


Map(center=[39.41956382721997, -0.431725495619315], controls=(WidgetControl(options=['position', 'transparent_…

# 6.导出对照方法结果


In [19]:
# --- 1. 定义通用的导出参数 ---

# 定义一个统一的文件夹名，所有结果都会保存在这里
# 使用目标日期命名，方便归档
folder_name = f"Flood_Comparison_Results_{targdate}"

# 定义导出分辨率 (米)
export_scale = 10 

# 定义导出区域
export_region = roi_choose1 # 使用.geometry()确保传递的是纯几何对象

print(f"所有文件将被导出到Google Drive的 '{folder_name}' 文件夹中。")
print("请稍后前往GEE Code Editor的 'Tasks' 标签页手动运行所有任务。")

# --- 2. 创建一个待导出影像的字典 ---
# 这样做代码更整洁，方便管理
# 键(key)是文件名，值(value)是影像对象
image_exports = {
    'Flood_Map_Otsu_JRC': 'flood_map_otsu',
    'Flood_Map_KMeans_JRC': 'flood_map_kmeans',
    'Flood_Map_ZScore_JRC': 'flood_map_zscore'
}

# --- 3. 循环提交导出任务 ---

for filename, image_var_name in image_exports.items():
    
    # 检查影像变量是否存在，以防某个方法之前运行失败
    if image_var_name in globals():
        
        # 从全局变量中获取影像对象
        image_to_export = globals()[image_var_name]
        
        # 准备任务
        task = ee.batch.Export.image.toDrive(
            # 将二值影像转为Byte类型，可以极大减小文件体积
            image=image_to_export.toByte().clip(export_region), 
            description=f'Export_{filename}', # GEE Tasks面板中显示的任务名
            folder=folder_name,
            fileNamePrefix=filename, # 最终的文件名
            region=export_region,
            scale=export_scale,
            fileFormat='GeoTIFF',
            maxPixels=1e13 
        )
        
        # 启动任务
        task.start()
        
        print(f"  > 成功提交影像 '{filename}.tif' 的导出任务。")
    
    else:
        print(f"  > 警告：未找到名为 '{image_var_name}' 的影像变量，跳过导出。")

所有文件将被导出到Google Drive的 'Flood_Comparison_Results_2024-10-31' 文件夹中。
请稍后前往GEE Code Editor的 'Tasks' 标签页手动运行所有任务。
  > 成功提交影像 'Flood_Map_Otsu_JRC.tif' 的导出任务。
  > 成功提交影像 'Flood_Map_KMeans_JRC.tif' 的导出任务。
  > 成功提交影像 'Flood_Map_ZScore_JRC.tif' 的导出任务。
